Property finder relases all the data 

<h3>Scrapping all ad pages  </h3>

<h5>When using Headless option you have to add user agent 
 because Headless mode uses its own default User-Agent if it is not given as an argument.
 However some webpages may block Headless mode User-Agent to avoid unwanted traffic.
 It may result in Access denied error while trying to open a webpage. </h5>

In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('Property_finder_ad_pages_final.csv', 'w') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([[link] for link in ads_pages])

chrome.quit()


Set Options

Scrapping all properties 

In [ ]:
# Get all Home pages links
def scrapp_all_properties():
    page_links = [f'https://www.propertyfinder.eg/en/buy/properties-for-sale.html?page={i}' for i in range(1, 4000)]

    chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))
    counter=0
    ads_pages = []
    total_ads_scrapped = 0

    # Send a GET request to the webpage
    for i in page_links:
        try:
            chrome.get(i)
        except TimeoutException as e:
            print(f"Failed to load page: {i}. Error: {e}")
            continue
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(chrome.page_source, 'html.parser')
        # Find and print all links on the page
        links = soup.select('a.card__link')
        ads_pages.extend(links)
        print(f'links visited:{chrome.current_url.split("=")[1]}')
        counter=counter+1
        if counter==5:
            break
    print(f'Total Ads Scrapped:{len(ads_pages)}')
    #Convert it to set to remove duplicated links
    set(ads_pages)
    home_page_all_ads=set(ads_pages)
    # joblib.dump(ads_pages, 'state2.pkl')
    # This line is creating two empty lists, 'href' and 'title', to which we will append values in the loop below.
    # This line iterates over each element 'x' in the list 'ads_pages'. 
    #!For each element x it extracts the 'href' attribute and appends it to the 'href' list, and also extracts the 'title' attribute and appends it to the 'title' list. 
    #!The results are stored as tuples inside the respective lists.


    # Create an empty deque
    href_deque = deque()

    # Use list comprehension to append items to deque
    ([(href_deque.appendleft(x.get('href'))) for x in ads_pages])
    joblib.dump(href_deque, 'href_deque_state.pkl')



    # Open a new CSV file for writing
    # with open('Property_finder_ad_pages.csv', 'wb') as f:
    #     # Create a CSV writer object
    #     writer = csv.writer(f)
    #     # Write each link to a new row in the file
    #     writer.writerows([str(link)] for link in ads_pages)
    chrome.quit()
  
scrapp_all_properties()

<h4>Save to CSV and check all scrapped</h4>

In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('All_properties_property_finder_ad_pages.csv', 'wb') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([str(link)] for link in ads_pages)

chrome.quit()

<h3>Set options<h3>

In [6]:
import csv
from math import ceil
from random import sample
from time import sleep
import requests
from bs4 import BeautifulSoup
from requests import get
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd 
import numpy as np 
from collections import deque
import joblib
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

def set_options(*args):
    options = Options()
    prefs = {'profile.managed_default_content_settings.images': 2}
    options.add_experimental_option('prefs', prefs)
    
    for arg in args:
        options.add_argument(arg)
        
    #! eager : This strategy causes Selenium to wait for the DOMContentLoaded event (html content downloaded and parsed only).
    #!we will use eager because the items we scrapping is loading fast 
    options.page_load_strategy = 'eager'
    return options

<h4>Scrapp Ad Pages </h4>

In [2]:
# Create an empty list to store the amenities
#DONE add joblib Done
#DONE add deqeue Done
#DONE save all variables into dictionary to dump it to pickle with joblib  Done

amenities_list = []
# ads_list_dic=[]


chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))
try:    
    href_deque = joblib.load('href_deque_state.pkl')
    ads_list_dic=joblib.load('ads_scrapped.pkl')
except:
    #FIXME not logic to scrapp all again 
    # scrapp_all_properties()
    href_deque = joblib.load('href_deque_state.pkl')

    
finally:
    
    while href_deque:
        #it will pop from the left (newest added) to the oldest 
        # href_deque.popleft()
        current_url = href_deque.popleft()
        ad={'url':current_url}

        print(current_url)
        scraped_urls = {x['url'] for x in ads_list_dic}
        
        if current_url not in scraped_urls:
            print(len(ads_list_dic), current_url)
            while True:
                    try:
                        chrome.get(current_url)
                        #sleep(2.0)
                        break
                    except TimeoutException:
                        print('timeout detected... cooling off...')
                        sleep(2.0)
                        # chrome.get(href_deque[0])
                        #TODO make this dynamic 
                        
            soup = BeautifulSoup(chrome.page_source, 'html.parser')
            # Find and print subtitle of the page
            #!####################################################
            ad['title'] = soup.find('h1',class_ = 'property-page__title').text.strip()
            ad['sub_title'] = soup.find('h2',class_ = 'property-page__sub-title').text.strip()
            li = soup.find_all('div',class_ = 'property-facts__value')
            ad['property_type']=soup.find_all('div',class_ = 'property-facts__value')[0].text.strip()
            #Get all text
            property_size_temp=soup.find_all('div',class_ = 'property-facts__value')[1].text.strip()
            #Extract SQM size
            ad['property_size']=re.sub(r'\D*(\d+)\D*',r'\1', li[1].text.strip()[15:]).strip()
            ad['bedrooms']=soup.find_all('div',class_ = 'property-facts__value')[2].text.replace('\n','').replace('\t', '').strip()
            ad['bathrooms']=soup.find_all('div',class_ = 'property-facts__value')[3].text.strip()
            ad['agent_name']=soup.find('h4',class_ = 'property-agent__name').text.strip()
            ad['agent_company']=soup.find('div',class_ = 'property-agent__position-broker-name').text.strip()
            ad['agent_number'] = soup.find('a', class_='property-contact__primary-button').get('href')[4:]
            ad['description']=soup.find('div',class_ = 'property-description__text-trim').text.replace('\n','').strip()
            ad['listed_date']=soup.find_all('div',class_ = 'property-page__legal-list-content')[1].text.strip()
            #! Ameneties 
            # Get all the divs with class 'property-amenities__list' from a BeautifulSoup object called soup
            amenities_all_divs = soup.find_all('div', class_='property-amenities__list')
            # Iterate over each div in the amenities_all_divs list and add its text content (with leading/trailing whitespace removed) to the amenities_list
            [amenities_list.append(x.text.strip()) for x in amenities_all_divs]
            # Join the elements of the amenities_list together into a single string, separated by semicolons (;), and assign the result to the amenities variable
            ad['amenities'] = ";".join(amenities_list)

            #Add Dictionary to list 
            ads_list_dic.append(ad)
            joblib.dump(ads_list_dic, 'ads_scrapped.pkl')
            joblib.dump(href_deque, 'href_deque_state.pkl')

            #pop first left link 
chrome.quit()

C:\Users\amigo\AppData\Local\Temp\ipykernel_38372\2768155591.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


https://www.propertyfinder.eg/en/plp/buy/chalet-for-sale-north-coast-sidi-abdel-rahman-marassi-3913189.html
10 https://www.propertyfinder.eg/en/plp/buy/chalet-for-sale-north-coast-sidi-abdel-rahman-marassi-3913189.html
https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-el-katameya-el-katameya-compounds-the-brooks-3856217.html
11 https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-el-katameya-el-katameya-compounds-the-brooks-3856217.html
https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-hyde-park-3732243.html
12 https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-hyde-park-3732243.html
https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-red-sea-hurghada-al-gouna-mangroovy-residence-3725705.html
13 https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-red-sea-hurghada-al-gouna-mangroovy-residen